In [ ]:
import tensorflow as tf
import keras
import numpy as np
import cv2
import os
from random import shuffle
import urllib.request
import imutils
from tqdm import tqdm
import pickle
import sklearn as sk
import matplotlib.pyplot as plt
import re
from datetime import datetime
from keras import backend as K

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping,TensorBoard,TerminateOnNaN

In [ ]:
cd C:\Users\Jordan\Desktop\N_game_images

In [ ]:
def store_raw_images():
    #'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n12830222'
    neg_images_link = 'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n03743016'
    neg_image_urls = urllib.request.urlopen(neg_images_link).read().decode()
    pic_num = 1
    
    img_path = r"C:\Users\Jordan\Desktop\N_game_images"
    
    for i in neg_image_urls.split('\n'):
        try:
            print(i)
            urllib.request.urlretrieve(i, img_path+'\\negatives\\negatives'+str(pic_num)+'.jpg')
            img = cv2.imread(img_path+'\\negatives\\negatives'+str(pic_num)+'.jpg',cv2.IMREAD_GRAYSCALE)
            # should be larger than samples / pos pic (so we can place our image on it)
            resized_image = cv2.resize(img, (100, 100))
            cv2.imwrite(img_path+'\\negatives\\negatives'+str(pic_num)+'.jpg',resized_image)
            pic_num += 1
            
        except Exception as e:
            print(str(e))  

In [ ]:
for file_type in [img_path]:
    for img in os.listdir(file_type+'\\originals'):
        positive = cv2.imread(img_path+'\\originals\\'+str(img))
        resize_positive = cv2.resize(positive,(50,50))
        cv2.imwrite(img_path+'\\positives\\'+str(img),resize_positive)
        

In [ ]:
for file_type in [img_path]:
    for img in os.listdir(file_type+'\\positives'):
        positive = cv2.imread(img_path+'\\positives\\'+str(img))
        for angle in np.arange(5,360,5):
            rotated = imutils.rotate_bound(positive, angle)
            cv2.imwrite(img_path+'\\positives\\'+str(img)[:-4]+'_'+str(angle)+'.JPG',rotated)
            

In [2]:
img_path = r"C:\Users\Jordan\Desktop\N_game_images"

In [ ]:
mean = 0
for file_type in tqdm([img_path]):
    for img in os.listdir(file_type+'\\originals'):
        positive = cv2.imread(img_path+'\\originals\\'+str(img))
        for angle in np.arange(5,360,10):
            rotated = imutils.rotate_bound(positive, angle)
            cv2.imwrite(img_path+'\\originals_rotated\\'+str(img)[:-4]+'_'+str(angle)+'.JPG',rotated)
            for s in np.arange(20,600,20):
                gaus = cv2.randn(rotated,mean,int(s))
                cv2.imwrite(img_path+'\\originals_rotated\\'+str(img)[:-4]+'_'+str(angle)+'_gauss_sd_'+str(s)+'.JPG',gaus)

In [ ]:
train_dir = img_path + "\\originals_rotated"
test_dir = img_path + "\\originals"

In [ ]:
img_size = 60

In [ ]:
def original_labels(images_path):
    original_labels_list = []
    for file_type in [img_path]:
        for img in os.listdir(file_type+'\\originals'):
            original_labels_list.append(img.split('.')[0])
            
        return original_labels_list

"""Convolution Neural Networks require a binary array the 
length of the number of images needed to be classified
as a label, with 1 corresponding to image of interest"""
def label_img(image,labels_list):
    
    #merge repeated labels
    new_labels = labels_list
    for i,label in enumerate(labels_list):
        new_labels[i] = re.sub('[0-9]+', '',label) #remove number from label
    new_labels = list(set(new_labels))
    new_labels.sort()

    for output,label in enumerate(new_labels):
        word_label = image.split('.')[0]
        
        if label in word_label:
            label_array= keras.utils.to_categorical(output,num_classes=len(new_labels)) #populates image position with 1
            return label_array

In [ ]:
labels= original_labels(test_dir)

merged_labels = labels
for i,label in enumerate(labels):
     merged_labels[i] = re.sub('[0-9]+', '',label) #remove number from label
merged_labels = list(set(merged_labels))
merged_labels.sort()

In [ ]:
def create_train_data():
    training_data = []
    training_labels = []
    for img in tqdm(os.listdir(train_dir)):
        label = label_img(img,labels)
        path = os.path.join(train_dir,img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img_size,img_size))
        img = img.reshape([img_size,img_size,1])
        training_data.append(img)
        training_labels.append(label) #use for sentdex tutorial
    
    training_data,training_labels= sk.utils.shuffle(training_data,training_labels)
    training_data = np.array(training_data)
    
    #training_data = training_data.reshape(training_data.shape[0],img_size,img_size,1)
    np.save('train_data.npy', training_data)
    np.save('train_labels.npy',training_labels)
    return training_data,training_labels

In [ ]:
#training_data,training_labels = create_train_data()
training_data = np.load('C:\\Users\\Jordan\\Desktop\\N_game_images\\train_data.npy',allow_pickle=True,encoding='latin1')
training_labels = np.load('C:\\Users\\Jordan\\Desktop\\N_game_images\\train_labels.npy',allow_pickle=True,encoding='latin1')

In [ ]:
def process_test_data():
    testing_data = []
    testing_labels = []
    for img in tqdm(os.listdir(test_dir)):
        label = label_img(img,labels)
        path = os.path.join(test_dir,img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)       
        img = cv2.resize(img,(img_size,img_size))
        img = img.reshape([img_size,img_size,1])
        testing_data.append(img) #use for sentdex tutorial
        testing_labels.append(label)
        
        
    #testing_data,testing_labels= sk.utils.shuffle(testing_data,testing_labels)
    testing_data = np.array(testing_data)
    
    
    np.save('test_data.npy', testing_data)
    np.save('test_labels.npy',testing_labels)
    return testing_data,testing_labels

In [ ]:
#testing_data,testing_labels = process_test_data()
testing_data = np.load('test_data.npy',allow_pickle=True,encoding='latin1')
testing_labels = np.load('test_labels.npy')

In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
conv_node_num = 64
learning_rate = 1e-3
full_node = [4096] #4096 working best

for node in full_node:
    
    keras.backend.clear_session()

    tf.reset_default_graph()

    model_name = 'n_game_objects-{}-{}-64-{}.h5'.format(learning_rate, '6conv-basic',node)

    if os.path.exists('{}.meta'.format(model_name)):
        keras.models.load_model(model_name)
        print('model loaded!')
    else:
        convnet = tflearn.input_data(shape=[None, img_size, img_size, 1], name='input')

        convnet = conv_2d(convnet, 64, 5, activation='relu')
        convnet = max_pool_2d(convnet, 5)
        
        convnet = conv_2d(convnet, 64, 5, activation='relu')
        convnet = max_pool_2d(convnet, 5)

        convnet = conv_2d(convnet, 64, 5, activation='relu')
        convnet = max_pool_2d(convnet, 5)

        convnet = conv_2d(convnet, 64, 5, activation='relu')
        convnet = max_pool_2d(convnet, 5)

        convnet = conv_2d(convnet, 64, 5, activation='relu')
        convnet = max_pool_2d(convnet, 5)

        convnet = conv_2d(convnet, 64, 5, activation='relu')
        convnet = max_pool_2d(convnet, 5)

        convnet = fully_connected(convnet, node, activation='relu')
        convnet = dropout(convnet, 0.8)

        convnet = fully_connected(convnet, 30, activation='softmax') #output layer, second number corresponds to number of outputs
        convnet = regression(convnet, optimizer='adam', 
                             learning_rate=learning_rate, loss='categorical_crossentropy', name='targets')

        model = tflearn.DNN(convnet, tensorboard_dir = 'log')


        train = training_data[:-500]

        test = training_data[-500:]

        X = np.array([i[0] for i in train]).reshape(-1,img_size,img_size,1)
        Y = [i[1] for i in train]

        test_x = np.array([i[0] for i in test]).reshape(-1,img_size,img_size,1)
        test_y = [i[1] for i in test]
        
        print(model_name)

        model.fit({'input': X}, {'targets': Y}, n_epoch=1, validation_set=({'input': test_x}, {'targets': test_y}), 
        snapshot_step=500, show_metric=True, run_id=model_name)

In [ ]:
training_data[0][1]

In [ ]:

#keras.models.save_model(model_name)

In [ ]:
model.save(model_name)

In [ ]:
keras.models.load_model(model_name)

In [ ]:
#tensorboard --logdir=log/ --host localhost --port 8088

In [ ]:
callbacks=[keras.callbacks.TensorBoard(log_dir="log\\{}".format(model_name), 
                                                     histogram_freq=1, write_graph=True, write_images=True)]

# Keras tutorial settings

In [ ]:
#normalize training and testing data
training_data = training_data.astype('float32')
testing_data = testing_data.astype('float32')
training_dataNorm = training_data/255
testing_dataNorm = testing_data/255

In [ ]:
plt.figure(figsize = (10,10))
for i in range(testing_data.shape[0]):
    plt.subplot(6,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(merged_labels[np.argmax(testing_labels[i])])
    plt.imshow(np.squeeze(testing_data[i]), cmap = 'gray', interpolation = 'bicubic')
    plt.colorbar()
plt.show()

In [ ]:
# Returns a convolution neural network model
def create_CNN(input_data,cnn_nodes = 32, full_nodes = 512, output_nodes = 30,
                 hidden_layers = 6,fully_connected = 1,reg=0.0005,learn_rate=1e-4):
    
    #have to always clear keras session whenever running a new model
    keras.backend.clear_session()
    tf.compat.v1.reset_default_graph()
    
    reg = keras.regularizers.l2(reg)
    
    #check dimensions of input data, must correspond to that of one image (3D)
    if np.ndim(input_data) > 3:
        input_data = input_data[1] #removes four dimension
    
    model = Sequential()

    #input layer
    model.add(Conv2D(cnn_nodes, (3, 3), kernel_regularizer = reg,
                     input_shape=([60,60,1])))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    
    multiplier = 2
    for iLayers in range(hidden_layers):
        #hidden layers
        model.add(Conv2D(cnn_nodes*multiplier, (3, 3), kernel_regularizer = reg))
        model.add(BatchNormalization(axis=-1))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        multiplier += 2 #powers of 2 increasing hidden layer CNN size


    model.add(Flatten())

    # Fully connected layer
    model.add(Dropout(0.25))
    model.add(Dense(full_nodes))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(output_nodes))

    model.add(Activation('softmax'))
    
    model.compile(optimizer=Adam(lr=learn_rate, decay = 1e-6),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    model_name = 'N_game_'+str(cnn_nodes)+'cnnStart_'+str(full_nodes)+'full_'+str(hidden_layers)+'hidden_'+str(len(model.layers))+'TotalLayers.h5'

    return model,model_name

In [ ]:
kmodel,kmodel_name = create_CNN(testing_data,cnn_nodes=64,full_nodes=512,
                                hidden_layers=3,output_nodes=len(testing_labels[0]),learn_rate=1e-3)
model_name = img_path+'\\models\\'+kmodel_name

In [ ]:
test_labels = np.array(testing_labels)

In [ ]:
def train_model(model,model_path,num_ep=5,train_data=True,use_generator=False):
    
    model_name = model_path.split('\\')[-1] # get model name from designated save path
    
    logdir = '.\\log\\'+model_name
    
    checkpoint_dir = img_path+"\\check_points\\" # get checkpoint dir
    
    #If using image generator to augment data
    if use_generator is True:
        model_path = model_path.split('.')[0]+'_Generator.'+model_path.split('.')[1]
        
    # Establish callbacks for model
    checkpoint_path = checkpoint_dir+model_name+"weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
    
    cp_callback = ModelCheckpoint(checkpoint_path,monitor='val_acc',save_best_only=True,verbose=0,mode='auto')
    early_stop = EarlyStopping(monitor='val_acc', min_delta=0.01, patience=3, verbose=1, 
                                              mode='auto', baseline=None, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                  patience=2, min_lr=1e-11)
    nan_term = TerminateOnNaN()

    tensorboard = TensorBoard(log_dir = logdir,histogram_freq=0,write_graph=False,
                                             write_grads=True,update_freq='epoch')

    callback_list = [cp_callback,early_stop,reduce_lr,nan_term,tensorboard]
    
    
    # check if model already exists. If it does, load weights of best epoch and continue training
    epNum = []
    epVal_acc = []
    if os.path.exists('{}'.format(model_path)):
        for cp in os.listdir(checkpoint_dir):
            if model_name in cp:
                epNum.append(int(cp[-12:-10]))
                epVal_acc.append(float(cp[-9:-5]))
        if epVal_acc:
            best_ep = epNum[np.argmax(epVal_acc)]
            model.load_weights(checkpoint_path.format(epoch=best_ep,val_acc=max(epVal_acc)))

            num_ep = best_ep+num_ep

            print('Weights loaded!')
        else: 
            best_ep = 0
    else:
        best_ep = 0 # If model not found will train from scratch
        if train_data is True:
            print('No version of model found, proceeding to train new model from scratch...')
        else: 
            print('No version of model found.')

    if train_data is True:
        if use_generator is True:
            model.fit_generator(train_generator,epochs=num_ep,initial_epoch=best_ep,
                                 steps_per_epoch=train_generator.n//train_generator.batch_size,
                                 validation_data = validation_generator,
                                 validation_steps = validation_generator.n//validation_generator.batch_size,
                                 verbose=1,callbacks=callback_list)
        else:
            model.fit(x=training_dataNorm,y=np.array(training_labels),epochs=num_ep,verbose=1,shuffle=True,
                       validation_data = (testing_dataNorm,test_labels), initial_epoch=best_ep,
                       callbacks=callback_list)
    model.save(model_path)
    
    return model

In [ ]:
kmodel_trained=train_model(kmodel,model_name,num_ep=5,use_generator=True)

In [ ]:
test_loss,test_acc = kmodel_trained.evaluate(testing_dataNorm, y=np.array(testing_labels),verbose=1)

In [ ]:
def custom_results_plot(model):
    history = model.history.history
    x = model.history.epoch
    plt.style.use("ggplot")
    plt.figure(figsize=(10,10))
    #plt.subplot(1,2,1)
    plt.plot(x,history["loss"],'r',label="train_loss")
    plt.plot(x,history["acc"],'g',label="train_acc")
    plt.plot(x,history["val_loss"],'r--',label="val_loss")
    plt.plot(x,history["val_acc"],'g--',label="val_acc")
    plt.title("Training/Val Loss and Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    
    #plt.subplot(1,2,2)
    #keras.utils.plot_model(kmodel)
    plt.show()

In [ ]:
custom_results_plot(kmodel_trained)

In [ ]:
# Get output shape of specific layers

import six
def get_activations(img, model, layer, batch_size=128):
    """
    Return the output of the specified layer for input `x`. `layer` is specified by layer index (between 0 and
    `nb_layers - 1`) or by name. The number of layers can be determined by counting the results returned by
    calling `layer_names`.
    :param x: Input for computing the activations.
    :type x: `np.ndarray`. Example: x.shape = (80, 80, 3)
    :param model: pre-trained Keras model. Including weights.
    :type model: keras.engine.sequential.Sequential. Example: model.input_shape = (None, 80, 80, 3)
    :param layer: Layer for computing the activations
    :type layer: `int` or `str`. Example: layer = 'flatten_2'
    :param batch_size: Size of batches.
    :type batch_size: `int`
    :return: The output of `layer`, where the first dimension is the batch size corresponding to `x`.
    :rtype: `np.ndarray`. Example: activations.shape = (1, 2000)
    """   
    layer_names = [layer.name for layer in model.layers]
    if isinstance(layer, six.string_types):
        if layer not in layer_names:
            raise ValueError('Layer name %s is not part of the graph.' % layer)
        layer_name = layer
    elif isinstance(layer, int):
        if layer < 0 or layer >= len(layer_names):
            raise ValueError('Layer index %d is outside of range (0 to %d included).'
                             % (layer, len(layer_names) - 1))
        layer_name = layer_names[layer]
    else:
        raise TypeError('Layer must be of type `str` or `int`.')

    layer_output = model.get_layer(layer_name).output
    layer_input = model.input
    output_func = K.function([layer_input, K.learning_phase()], [layer_output])

    # Apply preprocessing
    if img.shape == K.int_shape(model.input)[1:]:
        x_preproc = np.expand_dims(img, 0)
    else:
        x_preproc = img
    assert len(x_preproc.shape) == 4

    # Determine shape of expected output and prepare array
    output = output_func([x_preproc,1])[0]
    output_shape = output_func([x_preproc[0][None, ...]])[0].shape
    activations = np.zeros((x_preproc.shape[0],) + output_shape[1:], dtype=np.float32)

    # Get activations with batching
    for batch_index in range(int(np.ceil(x_preproc.shape[0] / float(batch_size)))):
        begin, end = batch_index * batch_size, min((batch_index + 1) * batch_size, x_preproc.shape[0])
        activations[begin:end] = output_func([x_preproc[begin:end]])[0]

    return output